In [ ]:
import numpy as np
import xml.etree.ElementTree as ET
import os
import pandas as pd
import json
import scipy.stats

In [ ]:
def dist_to_px(dist, spacing):
    """distance in um (or rather same unit as the spacing)"""
    dist_px = int(round(dist / spacing))
    return dist_px


def mm2_to_px(mm2, spacing):
    return (mm2 * 1e6) / spacing**2


def px_to_mm(px: int, spacing: float):
    return px * spacing / 1000


def px_to_um2(px, spacing):
    area_um2 = px * (spacing**2)
    return area_um2


def get_mask_area(slide, spacing=32):
    """Get the size of a mask in pixels where the mask is 1."""
    mask = np.load(slide)
    counts = np.unique(mask, return_counts=True)
    down = 6
    area = counts[1][1] * down**2
    return area


def points_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    return len(root[0][0][0])


def create_til_score(xml_path, bulk_path):
    """slide_level_nms"""

    cell_count = points_from_xml(xml_path)
    # print(cell_count)

    """Compute TIL score and write to output"""
    til_area = dist_to_px(4, 0.5) ** 2
    tils_area = cell_count * til_area
    stroma_area = get_mask_area(bulk_path)
    tilscore = int((100 / int(stroma_area)) * int(tils_area))
    tilscore = min(100, tilscore)
    tilscore = max(0, tilscore)
    print(f"tilscore = {tilscore}")
    return tilscore

In [ ]:
xml_path = "/home/u1910100/GitHub/TIAger-Torch/xml/105S.xml"
bulk_path = "/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/105S_bulk.npy"
create_til_score(xml_path, bulk_path)

In [ ]:
ground_truth = pd.read_csv(
    "/home/u1910100/Documents/Tiger_Data/wsitils/tiger-til-scores-wsitils.csv"
)
# print(ground_truth['image-id'], ground_truth['tils-score'])

til_score_dict = {}

for i, row in ground_truth.iterrows():
    id = row["image-id"]
    score = row["tils-score"]
    til_score_dict[id] = score

print(til_score_dict)

with open("ground_truth_tils.json", "w") as file:
    json.dump(til_score_dict, file)

In [ ]:
xml_dir = "/home/u1910100/GitHub/TIAger-Torch/xml"
pred_til_score_dict = {}

for xml_file in os.listdir(xml_dir):
    xml_path = os.path.join(xml_dir, xml_file)
    slide_name = os.path.splitext(xml_file)[0]

    bulk_path = (
        f"/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/{slide_name}_bulk.npy"
    )
    if not os.path.exists(bulk_path):
        bulk_path = f"/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/{slide_name}_seg.npy"
    if not os.path.exists(bulk_path):
        continue
    score = create_til_score(xml_path, bulk_path)

    pred_til_score_dict[slide_name] = score

print(pred_til_score_dict)

In [29]:
print(len(pred_til_score_dict))

pred_tils = []
true_tils = []
for slide_id in pred_til_score_dict.keys():
    pred_tils.append(pred_til_score_dict[slide_id])
    true_tils.append(til_score_dict[slide_id])

print(scipy.stats.pearsonr(true_tils, pred_tils))

81
PearsonRResult(statistic=0.6958452429233689, pvalue=5.580753248587816e-13)
